# A Gentle Introduction to Ray Core by Example

Many of Ray’s core concepts can be explained with a good example,
so that’s exactly what we’ll do in this example.
To run this example on your own, make sure you have Ray installed:

In [ ]:
! pip install ray

## A Ray Core Intro

To start a local cluster you can simply run

In [ ]:
import ray
ray.init()

After running this code you will see output of the following form.
We omit a lot of information in this example output, as that
would require you to understand more of Ray’s internals first.

```
... INFO services.py:1263 -- View the Ray dashboard at http://127.0.0.1:8265
{'node_ip_address': '192.168.1.41',
...
'node_id': '...'}
```

The output of this command indicates that your Ray cluster is working properly.
As shown in the first line of the output, Ray includes its own dashboard that can be accessed at `http://127.0.0.1:8265`
 (unless a different port is listed in the output). You can take some time to explore the dashboard,
 which will display information such as the number of CPU cores available and the total utilization of your current Ray application.
 If you want to see the resource utilization of your Ray cluster within a Python script, you can use the `ray.cluster_resources()` function.
 Here's a realistic output that you might see on your laptop:

```
{'CPU': 12.0,
'memory': 14203886388.0,
'node:127.0.0.1': 1.0,
'object_store_memory': 2147483648.0}
```

Next, we want to give you a brief introduction to the Ray Core API, which we simply will refer to as the Ray API from now on.
One of the great things about the Ray API for Python programmers is that it feels very familiar,
using concepts such as decorators, functions, and classes.
The Ray API is designed to provide a universal programming interface for distributed computing.
The Ray engine handles the complicated work behind the scenes, allowing Ray to be used with existing Python libraries and systems.

We want to give you an in-depth understanding of how Ray works and how you can grasp its basic concepts.
It is important to note that if you are a Python programmer with less experience or prefer to concentrate on more advanced tasks,
it may take some time to become familiar with Ray Core.
However, we highly recommend taking the time to learn the Ray Core API as it is a great way to start working with distributed computing using Python.

## Your First Ray API Example

To give you an example, take the following function which retrieves and processes
data from a database. Our dummy `database` is a plain Python list containing the
words of the title of the ["Learning Ray" book](https://www.amazon.com/Learning-Ray-Flexible-Distributed-Machine/dp/1098117220/).
To simulate the idea that accessing and processing data from the database is costly, we have the function
`sleep` (pause for a certain amount of time) in Python.

In [2]:
import time

database = [
    "Learning", "Ray",
    "Flexible", "Distributed", "Python", "for", "Machine", "Learning"
]


def retrieve(item):
    time.sleep(item / 10.)
    return item, database[item]

Our database has eight items in total. If we were to retrieve all items sequentially,
how long should that take?
For the item with index 5 we wait for half a second `(5 / 10.)` and so on.
In total, we can expect a runtime of around `(0+1+2+3+4+5+6+7)/10. = 2.8` seconds.
Let’s see if that’s what we actually get:

In [3]:
def print_runtime(input_data, start_time):
    print(f'Runtime: {time.time() - start_time:.2f} seconds, data:')
    print(*input_data, sep="\n")


start = time.time()
data = [retrieve(item) for item in range(8)]
print_runtime(data, start)

Runtime: 2.82 seconds, data:
(0, 'Learning')
(1, 'Ray')
(2, 'Flexible')
(3, 'Distributed')
(4, 'Python')
(5, 'for')
(6, 'Machine')
(7, 'Learning')


The total time it takes to run the function is 2.82 seconds, but this may vary on your individual computer.
It's important to note that our basic Python version is not capable of running this function simultaneously.

You may not have been surprised to hear this, but it's likely that you at least suspected
that Python list comprehensions are more efficient in terms of performance.
The runtime we measured, which was 2.8 seconds, is actually the worst case scenario.
It may be frustrating to see that a program that mostly just "sleeps" during its runtime could still be so slow,
but the reason for this is due to the Global Interpreter Lock (GIL), which gets enough criticism as it is.

### Ray Tasks

It’s reasonable to assume that such a task can benefit from parallelization.
Perfectly distributed, the runtime should not take much longer than the longest subtask,
namely `7/10. = 0.7` seconds.
So, let’s see how you can extend this example to run on Ray.
To do so, you start by using the @ray.remote decorator

In [5]:
import ray 


@ray.remote
def retrieve_task(item):
    return retrieve(item)

In this way, the function retrieve_task becomes a so-called _Ray task_.
In essence, a Ray task is a function that gets executed on a different process that
it was called from, potentially on a different machine.

It is very convenient to use Ray because you can continue to write your Python code as usual,
without having to significantly change your approach or programming style.
Using the `@ray.remote` decorator on your retrieve function is the intended use of decorators,
but for the purpose of clarity, we did not modify the original code in this example.

To retrieve database entries and measure performance, what changes do you need to make to the code?
It's actually not a lot. Here's an overview of the process:

In [6]:
start = time.time()
object_references = [
    retrieve_task.remote(item) for item in range(8)
]
data = ray.get(object_references)
print_runtime(data, start)

2022-12-20 13:52:34,632	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Runtime: 4.54 seconds, data:
(0, 'Learning')
(1, 'Ray')
(2, 'Flexible')
(3, 'Distributed')
(4, 'Python')
(5, 'for')
(6, 'Machine')
(7, 'Learning')


Have you noticed the differences?
To execute your Ray task remotely, you must use a `.remote()` call.
When tasks are executed remotely, even on your local cluster, Ray does it asynchronously.
The items in the object_references list in the previous code snippet do not directly contain the results.
In fact, if you check the Python type of the first item using `type(object_references[0])`,
you will see that it is actually an `ObjectRef`.
These object references correspond to _futures_ that you need to request the result of.
This is what the call to `ray.get(...)` is for. Whenever you call remote on a Ray task,
it will immediately return one or more object references.
You should consider Ray tasks as the primary way of creating objects.
In the following section, we will provide an example that links multiple tasks together and allows
Ray to handle passing and resolving the objects between them.

We'll continue working on this example, but let's take a moment to review what we have done so far.
You began with a Python function and decorated it with `@ray.remote`, which made the function a Ray task.
Instead of directly calling the original function in your code, you called `.remote(...)` on the Ray task.
The final step was to retrieve the results back from your Ray cluster using `.get(...)`.
This process is so straightforward that you can probably create your own Ray task from another function without referring back to this example.
Why don't you give it a try now?

Coming back to our example, by using Ray tasks, what did we gain in terms of performance?
On most laptops the runtime clocks in at around 0.71 seconds,
which is just slightly more than the longest subtask, which comes in at 0.7 seconds.
That’s great and much better than before, but we can further improve our program by leveraging more of Ray’s API.

### The Object Store

One aspect that may have been noticed is that the retrieve definition involves directly accessing items from the `database`.
While this works well on a local Ray cluster, it is important to consider how this would function on an actual cluster with multiple computers.
In a Ray cluster, there is a head node with a driver process and multiple worker nodes with worker processes executing tasks.
By default, Ray creates as many worker processes as there are CPU cores on the machine.
However, in this scenario the database is defined on the driver only, but the worker processes need access to it to run the retrieve task.
Fortunately, Ray has a simple solution for sharing objects between the driver and workers or between workers - using
the `put` function to place the data into Ray's distributed object store.
In the `retrieve_task` definition, we explicitly include a `db` argument, which will later be passed the `db_object_ref` object.

In [7]:
db_object_ref = ray.put(database)


@ray.remote
def retrieve_task(item, db):
    time.sleep(item / 10.)
    return item, db[item]

By utilizing the object store in this manner, you can allow Ray to manage data access throughout the entire cluster.
Although interacting with the object store involves some overhead, it offers improved performance when dealing with larger, more realistic datasets.
For now, the crucial aspect is that this step is crucial in a truly distributed environment.
If desired, you can try rerunning the previous example with the new `retrieve_task` function to confirm that it still executes as expected.

### Non-blocking calls

In a previous example, we used `ray.get(object_references)` to retrieve results.
This call blocks the driver process until all results are available.
While this may not be an issue if the program finishes in a short amount of time, it could cause problems if each database item takes several minutes to process.
In this case, it would be more efficient to allow the driver process to perform other tasks while waiting for results,
and to process results as they are completed rather than waiting for all items to be finished.
Additionally, if one of the database items cannot be retrieved due to an issue like a deadlock in the database connection,
the driver will hang indefinitely.
To prevent this, it is a good idea to set reasonable timeouts using the wait function.
For example, if we do not want to wait longer than ten times the longest data retrieval task,
we can use the wait function to stop the task after that time has passed.

In [8]:
start = time.time()
object_references = [
    retrieve_task.remote(item, db_object_ref) for item in range(8)
]
all_data = []

while len(object_references) > 0:
    finished, object_references = ray.wait(
        object_references, num_returns=2, timeout=7.0
    )
    data = ray.get(finished)
    print_runtime(data, start)
    all_data.extend(data)

print_runtime(all_data, start)

Runtime: 0.11 seconds, data:
(0, 'Learning')
(1, 'Ray')
Runtime: 0.31 seconds, data:
(2, 'Flexible')
(3, 'Distributed')
Runtime: 0.51 seconds, data:
(4, 'Python')
(5, 'for')
Runtime: 0.71 seconds, data:
(6, 'Machine')
(7, 'Learning')
Runtime: 0.71 seconds, data:
(0, 'Learning')
(1, 'Ray')
(2, 'Flexible')
(3, 'Distributed')
(4, 'Python')
(5, 'for')
(6, 'Machine')
(7, 'Learning')


Instead of simply printing the results, we could have utilized the values that have been retrieved
within the `while` loop to initiate new tasks on other workers.

### Task dependencies

So far, our example program has been straightforward conceptually.
It involves one step, which is retrieving a group of items from a database.
However, let's say we want to perform an additional processing task on the data after it has been retrieved.
For example, we want to use the results from the first retrieval task to query other related data from the same database
(for example, from a different table).
The code below sets up this follow-up task and executes both the `retrieve_task` and `follow_up_task` in sequence.

In [9]:
@ray.remote
def follow_up_task(retrieve_result):
    original_item, _ = retrieve_result
    follow_up_result = retrieve(original_item + 1)
    return retrieve_result, follow_up_result


retrieve_refs = [retrieve_task.remote(item, db_object_ref) for item in [0, 2, 4, 6]]
follow_up_refs = [follow_up_task.remote(ref) for ref in retrieve_refs]

result = [print(data) for data in ray.get(follow_up_refs)]

((0, 'Learning'), (1, 'Ray'))
((2, 'Flexible'), (3, 'Distributed'))
((4, 'Python'), (5, 'for'))
((6, 'Machine'), (7, 'Learning'))


If you're not experienced with asynchronous programming, this example may not seem particularly impressive.
However, at second glance it might be somewhat surprising that the code runs at all.
So, what's the significance of this?
Essentially, the code appears to be a regular Python function with a few list comprehensions.

The point is that the function body of `follow_up_task` expects a Python tuple for its input argument `retrieve_result`.
However, when we use the `[follow_up_task.remote(ref) for ref in retrieve_refs]` command,
we are not actually passing tuples to the follow-up task at all.
Instead, we are using the `retrieve_refs` to pass in Ray object references.

Behind the scenes, Ray recognizes that the `follow_up_task` needs actual values,
so it will _automatically_ use the `ray.get` function to resolve these futures.
Additionally, Ray creates a dependency graph for all the tasks and executes them in a way that respects their dependencies.
This means that we don't have to explicitly tell Ray when to wait for a previous task to be completed -
it is able to infer this information on its own.
This feature of the Ray object store is particularly useful because it allows us to avoid copying large intermediate values
back to the driver by simply passing the object references to the next task and letting Ray handle the rest.

The next steps in the process will only be scheduled once the tasks specifically designed to retrieve information are completed.
In fact, if we had named the `retrieve_refs` task something like `retrieve_result`, you might not have even noticed this crucial detail.
This was intentional, as Ray wants you to concentrate on your work rather than the technicalities of cluster computing.
The dependency graph for the two tasks looks like this:

![Task dependency](https://raw.githubusercontent.com/maxpumperla/learning_ray/main/notebooks/images/chapter_02/task_dependency.png)

### Ray Actors

Before concluding this example, we should cover one more significant aspect of Ray Core.
As you can see in our example, everything is essentially a function.
We utilized the `@ray.remote` decorator to make certain functions remote, but besides that, we only used standard Python.

If we want to keep track of how often our database is being queried, we could just count the results of the retrieve tasks.
However, is there a more efficient way to do this? Ideally, we want to track this in a distributed manner that can handle a large amount of data.
Ray provides a solution with actors, which are used to run stateful computations on a cluster and can also communicate with each other.
Similar to how Ray tasks are created using decorated functions, Ray actors are created using decorated Python classes.
Therefore, we can create a simple counter using a Ray actor to track the number of database calls.

In [10]:
@ray.remote
class DataTracker:
    def __init__(self):
        self._counts = 0

    def increment(self):
        self._counts += 1

    def counts(self):
        return self._counts

The DataTracker class is an actor because it has been given the `ray.remote` decorator. This actor is capable of tracking state,
such as a counter, and its methods are Ray tasks that can be invoked in the same way as functions using `.remote()`.
We can modify the retrieve_task to incorporate this actor.

In [11]:
@ray.remote
def retrieve_tracker_task(item, tracker, db):
    time.sleep(item / 10.)
    tracker.increment.remote()
    return item, db[item]


tracker = DataTracker.remote()

object_references = [
    retrieve_tracker_task.remote(item, tracker, db_object_ref) for item in range(8)
]
data = ray.get(object_references)

print(data)
print(ray.get(tracker.counts.remote()))

[(0, 'Learning'), (1, 'Ray'), (2, 'Flexible'), (3, 'Distributed'), (4, 'Python'), (5, 'for'), (6, 'Machine'), (7, 'Learning')]
8


As expected, the outcome of this calculation is 8.
Although we didn't need actors to perform this calculation, it's valuable to have a way to maintain state across the cluster, possibly involving multiple tasks.
In fact, we could pass our actor into any related task or even into the constructor of a different actor.
The Ray API is very flexible, allowing for limitless possibilities.
It's also worth mentioning that it's not common for distributed Python tools to allow for stateful computations like this,
making it especially useful for running complex distributed algorithms such as reinforcement learning.
This concludes our detailed first example of the Ray API.
To wrap this up, let's briefly summarize the Ray API.

## A Summary of the Ray Core API

Looking back at what we did in the previous example, you will see that we only used six API methods.
These included `ray.init()` to initiate the cluster, `@ray.remote` to transform functions and classes into tasks and actors,
`ray.put()` to transfer values into Ray's object store, and `ray.get()` to retrieve objects from the cluster.
Additionally, we used `.remote()` on actor methods or tasks to execute code on our cluster, and `ray.wait` to prevent blocking calls.

The Ray API consists of far more than just these six calls, but knowing just these can get you quite far, if you're just starting out with Ray.
To make it easier to remember these methods, here's a summary:

- ray.init(): Initializes your Ray cluster. Pass in an address to connect to an existing cluster.
- @ray.remote: Turns functions into tasks and classes into actors.
- ray.put(): Puts values into Ray’s object store.
- ray.get(): Gets values from the object store. Returns the values you’ve put there or that were computed by a task or actor.
- .remote(): Runs actor methods or tasks on your Ray cluster and is used to instantiate actors.
- ray.wait(): Returns two lists of object references, one with finished tasks we’re waiting for and one with unfinished tasks.

## Want to learn more?

This example is a simplified version of the Ray Core walkthrough of [our "Learning Ray" book](https://maxpumperla.com/learning_ray/).
If you liked it, check out the [Ray Core Examples Gallery](./overview.rst) or some of the ML workloads in our [Use Case Gallery](../../ray-overview/use-cases.rst).